In [ ]:
import os
import pandas as pd
import cv2
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

data_dir = 'GTSRB/Training'
# /Users/abbylee/Downloads/GTSRB

In [ ]:
def preprocess(img):
    return cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)

In [1]:
import os
list_images = []
output = []

for dir in os.listdir(data_dir):
    i = 0
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(data_dir, dir)
    csv_file = pd.read_csv(os.path.join(inner_dir,"GT-" + dir + '.csv'), sep=';')
    for row in csv_file.iterrows():
        if i < 10:
            img_path = os.path.join(inner_dir, row[1].Filename)
            img = cv2.imread(img_path)
            img = img[row[1]['Roi.X1']:row[1]['Roi.X2'],row[1]['Roi.Y1']:row[1]['Roi.Y2'],:]
            img = preprocess(img)
            list_images.append(img)
            output.append(row[1].ClassId)
        i += 1

NameError: name 'data_dir' is not defined

In [ ]:
input_array = np.stack(list_images)
train_y = keras.utils.to_categorical(output)

randomize = np.arange(len(input_array))
np.random.shuffle(randomize)

x = input_array[randomize]
y = train_y[randomize]

validation: see how network is going

In [ ]:
split_size = int(x.shape[0]*0.6)
train_x, val_x = x[:split_size], x[split_size:]
train1_y, val_y = y[:split_size], y[split_size:]

split_size = int(val_x.shape[0]*0.5)
val_x, test_x = val_x[:split_size], val_x[split_size:]
val_y, test_y = val_y[:split_size], val_y[split_size:]

epoch: how many times we are iterating through data in a taining session.
batch_size: how many data used for once
sequential: determine architecture as one long list

conv2D(16, (3,3)kernal size, ...)
(3*3*16 stack up together)

MaxPooling2D: take small neighborhood pixel and takes the max of it (shrink down the pixel)

lr (learning rate)

In [ ]:
epochs = 10
batch_size = 16

input_shape = Input(shape=(32, 32,3))

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(64,64,3), padding='same'),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(units=2048, activation='relu'),
    Dense(units=1024, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=43, input_dim=2048, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
trained_model_conv = model.fit(train_x.reshape(-1,64,64,3), train1_y, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y))

In [ ]:
model.evaluate(test_x, test_y)